# Liver Model Construction: Glycolysis Gluconeogenesis & Lactate Fermentation

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.1


## Set paths and constants

### Directory paths

In [ ]:
model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

# Create MASS model from COBRA model

### Load COBRA model

In [10]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
recon=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core_model

Name,None
Memory address,0x07fdd0e0f8a60
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [11]:
glycogenolysis = MassModel("Glycogenolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [5]:
# glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
#                                 highlight_missing=True) 
# glycolysis_map

In [12]:
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
                'PYRt2m', 
              #  'SUCR', This is a sucrose metabolism reaction... 
                'G6PDH2r',
                'PGMT',
                'G6Pter',
                'GLCter',
                'TKT2',
                'PGCD', 
                '3SPYRSP', 
                'ALATA_L', 
                '2AMACHYD', 
                'PEPCK_re',
                'MDH',
                'MALtm', 
                'EX_glc__D_c',
                'EX_pyr_c',
                'EX_lac__L_c',
                'G6Pter',
                'G6PPer',
                'GLCter']



new = ['EX_glc__D_e', 'GLCt1', 'H2Oter', 'PIter', 'H2Ot', 'PIt', 'EX_pi_e', 
       'EX_h2o_e', 'EX_h_e', 'Ht'] #from RECONN


new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 

new_reaction_list
for i in new: 
    i = recon.reactions.get_by_id(i)
    new_reaction_list.append(i) 

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])
for rid in new:
    reaction = recon.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])  
new_reaction_list

metabolite = core_model.metabolites.get_by_id('g6p_c')
metabolite = core_model.metabolites.get_by_id('f6p_c')

SK_g6p_c = glycogenolysis.add_boundary(
    metabolite= 'g6p_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

SK_f6p_c = glycogenolysis.add_boundary(
    metabolite= 'f6p_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

SK_atp_c = glycogenolysis.add_boundary(
    metabolite= 'atp_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

SK_adp_c = glycogenolysis.add_boundary(
    metabolite= 'adp_c', boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

glycogenolysis.add_reactions([MassReaction(SK_g6p_c)]) 
new_reaction_list.append(SK_g6p_c)

print(new_reaction_list)

Ignoring reaction 'G6Pter' since it already exists.
Ignoring reaction 'GLCter' since it already exists.
Ignoring reaction 'SK_g6p_c' since it already exists.


[<Reaction HEX1 at 0x7fdd126b5160>, <Reaction PGI at 0x7fdd126b5dc0>, <Reaction FBP at 0x7fdd2b768880>, <Reaction PFK at 0x7fdd126b51c0>, <Reaction FBA at 0x7fdd126b5220>, <Reaction TPI at 0x7fdd126b50d0>, <Reaction GAPD at 0x7fdd10f4c340>, <Reaction PGK at 0x7fdd10f4c070>, <Reaction PGM at 0x7fdd21be8bb0>, <Reaction ENO at 0x7fdd21c1fc40>, <Reaction PEPtm at 0x7fdd2c144310>, <Reaction PEPCKm at 0x7fdd2b768b20>, <Reaction PYK at 0x7fdd13659e50>, <Reaction PCm at 0x7fdd2baa9e50>, <Reaction LDH_L at 0x7fdd2a990700>, <Reaction PYRt2m at 0x7fdd2c144460>, <Reaction G6PDH2r at 0x7fdd2a5cba30>, <Reaction PGMT at 0x7fdd2ad58580>, <Reaction G6Pter at 0x7fdd2b76e6d0>, <Reaction GLCter at 0x7fdd0e884eb0>, <Reaction TKT2 at 0x7fdd2b768f10>, <Reaction PGCD at 0x7fdd2771c790>, <Reaction 3SPYRSP at 0x7fdd2771c460>, <Reaction ALATA_L at 0x7fdd2bbbb040>, <Reaction 2AMACHYD at 0x7fdd2ba87670>, <Reaction PEPCK_re at 0x7fdd2ad58640>, <Reaction MDH at 0x7fdd2ad58400>, <Reaction MALtm at 0x7fdd0e8845e0>, <R

### View extracted network

In [7]:
escher_builder = escher.Builder(
    model=glycogenolysis,
    map_json=os.path.join(
        model_dir, ".".join((
            glycogenolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

ValueError: Could not load map_json. Not valid json, url, or filepath

## Parameterize MASS model

# Export MASS model

In [13]:
save_json_mass_model(
    mass_model=glycogenolysis,
    filename=os.path.join(model_dir, glycogenolysis.id + ".json"))